In [1]:
import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell
import random

In [2]:
random.seed(1337)
NBITS = 8
MAX_SEQ_LENGTH = 20
def gen_seq(_nseqs):
    zeros = [0] * NBITS
    ones = [1] * NBITS
    xs = []
    ys = []
    for _ in xrange(_nseqs):
        seq_len = random.randint(1, MAX_SEQ_LENGTH)
        # Note that we reserve the all-one vector as the delimeter symbol
        seq = [random.randint(0, 2**NBITS - 1) for _ in xrange(seq_len)]
        # Convert each int to binary
        seq = [
                [int(digit) for digit in '{0:08b}'.format(num)]
                for num in seq
              ]
        # Dummy inputs after the delimeter / outputs before the delieter
        dummies = [zeros] * seq_len
        x = seq + [ones] + dummies
        y = dummies + [ones] + seq
        xs.append(x)
        ys.append(y)

    return xs, ys

In [3]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = NBITS
n_hidden = 128 # hidden layer num of features
# As we need tensors which are "cubic"
# In reality, our sequences are of different lengths i.e. jagged
max_steps = 2*MAX_SEQ_LENGTH + 1

In [4]:
# tf Graph input

# Batch size, nsteps, ninput
x = tf.placeholder("float", [None, None, n_input])
istate = tf.placeholder("float", [None, 2*n_hidden]) #state & cell => 2x n_hidden
y = tf.placeholder("float", [None, n_input])
# Lengths of sequences
timesteps = tf.placeholder("int32")

# Define weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_input]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_input]))
}

In [5]:
def RNN(_X, _istate, _weights, _biases, _timesteps):
    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, n_input]) # (n_steps*batch_size, n_input)
    # Linear activation
    _X = tf.matmul(_X, _weights['hidden']) + _biases['hidden']

    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(0, max_steps, _X) # n_steps * (batch_size, n_hidden)

    # Get lstm cell output
    outputs, states = rnn.rnn(lstm_cell, _X,
                              initial_state=_istate,
                             sequence_length=_timesteps)

    # Linear activation
    # Get inner loop last output
    return [tf.matmul(out, _weights['out']) + _biases['out'] for out in outputs]

In [ ]:
# Loss function that averages cross entropy, taking into account
# the variable lengths of the sequences.
#
# _pred: list of tensors, where each element at index t is
#        batch_size * outputs at timestep t
def var_seq_loss(_preds, _y, _timesteps):
    #Get softmax_cross_entropy for each matri
    #tf.nn.softmax_cross_entropy_with_logits(pred, y)
    pass

In [6]:
pred = RNN(x, istate, weights, biases, timesteps)

# Define loss and optimizer
cost = var_seq_loss(pred, y, timesteps)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

TypeError: List of Tensors when single Tensor expected

In [ ]:
# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_xs = batch_xs.reshape((batch_size, n_steps, n_input))
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys,
                                       istate: np.zeros((batch_size, 2*n_hidden))})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys,
                                                istate: np.zeros((batch_size, 2*n_hidden))})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys,
                                             istate: np.zeros((batch_size, 2*n_hidden))})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + \
                  ", Training Accuracy= " + "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"
    # Calculate accuracy for 256 mnist test images
    test_len = 256
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print "Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label,
                                                             istate: np.zeros((test_len, 2*n_hidden))})